In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from os import chdir
chdir('/content/drive/MyDrive/IA')

In [3]:
import pandas as pd

In [ ]:
df = pd.read_excel('datasets/infomoney/infomoney_news_labelled.xlsx')
df

/usr/local/lib/python3.9/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,tweet,urls,text,label
0,Secretário da Fazenda diz que medidas do gover...,https://www.infomoney.com.br/politica/secretar...,Secretário da Fazenda diz que medidas do gover...,positiva
1,"Trade hoje: Mesmo acima dos 100 mil, Ibovespa ...",https://www.infomoney.com.br/mercados/trade-ho...,"Trade hoje: Mesmo acima dos 100 mil, Ibovespa ...",negativa
2,"Com alta de preços e crédito escasso, aumenta ...",https://www.infomoney.com.br/consumo/com-alta-...,"Com alta de preços e crédito escasso, aumenta ...",neutra
3,Criptos tem valor? Veja três mitos que confund...,https://www.infomoney.com.br/onde-investir/cri...,Criptos têm valor? Veja três mitos que confund...,NaN
4,“Pelo menos há agora um aceno à disciplina fis...,https://www.infomoney.com.br/economia/pelo-men...,“Pelo menos há agora um aceno à disciplina fis...,Neutra
...,...,...,...,...
18498,Auditoria do TCU não vê amparo legal em uso de...,https://www.infomoney.com.br/economia/auditori...,Auditoria do TCU não vê amparo legal em uso de...,NaN
18499,Ibovespa tem 5ª queda consecutiva e volta aos ...,https://www.infomoney.com.br/mercados/ibovespa...,Ibovespa tem 5ª queda consecutiva e volta aos ...,NaN
18500,Pessoas físicas vencem Wall Street e ação da G...,https://www.infomoney.com.br/mercados/pessoas-...,Pessoas físicas vencem Wall Street e ação da G...,NaN
18501,Sanofi produzirá mais de 100 milhões de doses ...,https://www.infomoney.com.br/economia/sanofi-p...,Sanofi produzirá mais de 100 milhões de doses ...,NaN


In [ ]:
df.drop(columns=['tweet', 'urls'], inplace=True)

In [ ]:
def lower_label(label):
    try:
        label = label.lower()
    except:
        return label
    return label

In [ ]:
df['label'] = df['label'].apply(lower_label)
df

,text,label
0,Secretário da Fazenda diz que medidas do gover...,positiva
1,"Trade hoje: Mesmo acima dos 100 mil, Ibovespa ...",negativa
2,"Com alta de preços e crédito escasso, aumenta ...",neutra
3,Criptos têm valor? Veja três mitos que confund...,NaN
4,“Pelo menos há agora um aceno à disciplina fis...,neutra
...,...,...
18498,Auditoria do TCU não vê amparo legal em uso de...,NaN
18499,Ibovespa tem 5ª queda consecutiva e volta aos ...,NaN
18500,Pessoas físicas vencem Wall Street e ação da G...,NaN
18501,Sanofi produzirá mais de 100 milhões de doses ...,NaN


In [ ]:
label_dict = {
    'positiva': 2,
    'neutra': 1,
    'negativa': 0.
}

def format_labels(label):
    if label in label_dict.keys():
        label = label_dict[label]
    else:
        label = None
    return label

In [ ]:
df['label'] = df['label'].apply(format_labels)
df

,text,label
0,Secretário da Fazenda diz que medidas do gover...,2.0
1,"Trade hoje: Mesmo acima dos 100 mil, Ibovespa ...",0.0
2,"Com alta de preços e crédito escasso, aumenta ...",1.0
3,Criptos têm valor? Veja três mitos que confund...,NaN
4,“Pelo menos há agora um aceno à disciplina fis...,1.0
...,...,...
18498,Auditoria do TCU não vê amparo legal em uso de...,NaN
18499,Ibovespa tem 5ª queda consecutiva e volta aos ...,NaN
18500,Pessoas físicas vencem Wall Street e ação da G...,NaN
18501,Sanofi produzirá mais de 100 milhões de doses ...,NaN


In [7]:
!pip install transformers huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.8 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch.nn.functional as F
import torch
import numpy as np

ModuleNotFoundError: ignored

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('trainings/bert-base-portuguese-cased_02')
tokenizer = AutoTokenizer.from_pretrained('trainings/bert-base-portuguese-cased_02')

In [ ]:
def predict_sentiment(input_text):
    input_tokens = tokenizer.encode_plus(input_text, return_tensors="pt", max_length=512, )

    input_tokens.to('cpu')

    with torch.no_grad():
        output = model(**input_tokens)
    
    logits = output.logits
    probabilities = F.softmax(logits, dim=-1)
    probabilities_np = probabilities.cpu().numpy()

    formatted_probabilities = np.array2string(probabilities_np, precision=6, suppress_small=True)

    predicted_class_idx = torch.argmax(probabilities, dim=1).item()

    '''
    return {
        'probabilities': formatted_probabilities,
        'predicted_class_idx': predicted_class_idx
    }
    '''
    return predicted_class_idx

In [ ]:
df['predict_label'] = None

In [ ]:
df.loc[:, 'predict_label'] = df.loc[:, 'text'].apply(predict_sentiment)

31373.409198760986 522.8901533126831 8.714835888544718


<ipython-input-68-90cf6a6c2d44>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'predict_label'] = df.loc[:, 'text'].apply(predict_sentiment)


In [ ]:
df

,text,label,predict_label
0,Secretário da Fazenda diz que medidas do gover...,2.0,2
1,"Trade hoje: Mesmo acima dos 100 mil, Ibovespa ...",0.0,0
2,"Com alta de preços e crédito escasso, aumenta ...",1.0,1
3,Criptos têm valor? Veja três mitos que confund...,NaN,1
4,“Pelo menos há agora um aceno à disciplina fis...,1.0,1
...,...,...,...
18498,Auditoria do TCU não vê amparo legal em uso de...,NaN,1
18499,Ibovespa tem 5ª queda consecutiva e volta aos ...,NaN,0
18500,Pessoas físicas vencem Wall Street e ação da G...,NaN,2
18501,Sanofi produzirá mais de 100 milhões de doses ...,NaN,2


In [ ]:
df.to_csv('infomoney_full_labelled.csv', index=False)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-92-a5170d346573>", line 1, in <cell line: 1>
    df.to_csv('infomoney_full_labelled.csv', index=False)
  File "/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/pandas/core/generic.py", line 3720, in to_csv
    return DataFrameRenderer(formatter).to_csv(
  File "/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/formats/format.py", line 1189, in to_csv
    csv_formatter.save()
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/formats/csvs.py", line 241, in save
    with get_handle(
  File "/usr/local/lib/python3.9/dist-pack

# Recuperando rotulagem feita anteriormente após erro

In [ ]:
predict_labels = ''
for i in range(df.shape[0]):
    print(df.loc[i, 'predict_label'], end=' ')
    predict_labels += str(df.loc[i, 'predict_label']) + ' '

2 0 1 1 1 1 0 1 0 0 1 0 0 2 0 1 2 2 1 1 1 1 0 0 0 2 2 1 1 1 1 2 2 1 1 1 0 0 2 1 1 0 1 2 2 0 1 1 0 0 2 2 1 1 2 2 2 1 1 2 0 1 2 1 0 0 0 2 0 1 2 0 2 2 0 1 1 1 2 1 1 2 2 2 1 1 1 1 0 0 1 1 1 0 1 0 1 2 1 2 1 1 1 2 2 1 0 2 0 1 0 0 0 1 0 1 1 1 1 1 2 1 2 1 0 0 1 0 0 1 1 0 2 1 0 0 1 0 1 2 1 0 0 1 1 0 0 2 1 1 2 1 0 1 2 1 2 1 1 1 1 0 0 1 1 0 1 0 1 2 1 1 1 2 0 0 2 2 2 2 2 0 2 1 2 1 2 2 2 2 1 2 1 1 2 1 0 1 1 0 1 0 1 1 1 0 2 2 0 1 0 1 1 0 2 2 0 1 1 1 1 0 1 2 1 0 1 0 2 1 1 1 0 1 2 0 1 2 1 0 1 2 2 1 2 0 1 0 2 2 2 0 2 2 0 1 1 2 0 0 2 0 1 1 1 2 1 1 1 0 0 0 1 2 0 2 0 1 2 0 2 1 1 0 1 2 2 1 2 1 2 1 1 2 2 2 0 1 1 1 2 1 1 0 0 0 1 1 2 1 1 1 0 1 2 1 0 2 2 1 1 0 1 1 2 0 1 2 2 0 0 1 1 2 1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 2 1 1 0 2 1 1 0 2 0 1 2 1 1 1 0 1 1 2 1 1 2 1 1 1 1 1 1 2 2 0 0 1 1 1 2 1 2 0 0 0 1 1 1 2 0 0 0 1 2 1 1 2 2 1 2 0 1 2 0 1 1 2 2 2 1 0 1 1 2 2 2 0 1 1 1 2 0 0 0 0 1 0 1 0 1 2 0 0 1 1 1 0 0 0 2 0 0 0 1 1 1 0 0 2 1 0 2 1 2 1 0 1 1 0 1 1 2 1 1 1 1 0 0 1 1 2 0 2 1 1 0 2 0 2 1 0 1 0 2 1 0 1 2 1 1 1 2 2 0 1 

In [ ]:
predict_labels

'2 0 1 1 1 1 0 1 0 0 1 0 0 2 0 1 2 2 1 1 1 1 0 0 0 2 2 1 1 1 1 2 2 1 1 1 0 0 2 1 1 0 1 2 2 0 1 1 0 0 2 2 1 1 2 2 2 1 1 2 0 1 2 1 0 0 0 2 0 1 2 0 2 2 0 1 1 1 2 1 1 2 2 2 1 1 1 1 0 0 1 1 1 0 1 0 1 2 1 2 1 1 1 2 2 1 0 2 0 1 0 0 0 1 0 1 1 1 1 1 2 1 2 1 0 0 1 0 0 1 1 0 2 1 0 0 1 0 1 2 1 0 0 1 1 0 0 2 1 1 2 1 0 1 2 1 2 1 1 1 1 0 0 1 1 0 1 0 1 2 1 1 1 2 0 0 2 2 2 2 2 0 2 1 2 1 2 2 2 2 1 2 1 1 2 1 0 1 1 0 1 0 1 1 1 0 2 2 0 1 0 1 1 0 2 2 0 1 1 1 1 0 1 2 1 0 1 0 2 1 1 1 0 1 2 0 1 2 1 0 1 2 2 1 2 0 1 0 2 2 2 0 2 2 0 1 1 2 0 0 2 0 1 1 1 2 1 1 1 0 0 0 1 2 0 2 0 1 2 0 2 1 1 0 1 2 2 1 2 1 2 1 1 2 2 2 0 1 1 1 2 1 1 0 0 0 1 1 2 1 1 1 0 1 2 1 0 2 2 1 1 0 1 1 2 0 1 2 2 0 0 1 1 2 1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 2 1 1 0 2 1 1 0 2 0 1 2 1 1 1 0 1 1 2 1 1 2 1 1 1 1 1 1 2 2 0 0 1 1 1 2 1 2 0 0 0 1 1 1 2 0 0 0 1 2 1 1 2 2 1 2 0 1 2 0 1 1 2 2 2 1 0 1 1 2 2 2 0 1 1 1 2 0 0 0 0 1 0 1 0 1 2 0 0 1 1 1 0 0 0 2 0 0 0 1 1 1 0 0 2 1 0 2 1 2 1 0 1 1 0 1 1 2 1 1 1 1 0 0 1 1 2 0 2 1 1 0 2 0 2 1 0 1 0 2 1 0 1 2 1 1 1 2 2 0 1

In [ ]:
df

,text,label,predict_label
0,Secretário da Fazenda diz que medidas do gover...,2.0,None
1,"Trade hoje: Mesmo acima dos 100 mil, Ibovespa ...",0.0,None
2,"Com alta de preços e crédito escasso, aumenta ...",1.0,None
3,Criptos têm valor? Veja três mitos que confund...,NaN,None
4,“Pelo menos há agora um aceno à disciplina fis...,1.0,None
...,...,...,...
18498,Auditoria do TCU não vê amparo legal em uso de...,NaN,None
18499,Ibovespa tem 5ª queda consecutiva e volta aos ...,NaN,None
18500,Pessoas físicas vencem Wall Street e ação da G...,NaN,None
18501,Sanofi produzirá mais de 100 milhões de doses ...,NaN,None


In [ ]:
with open('datasets/infomoney/predict_labels.txt', 'r') as predict_labels_file:
    predict_labels = predict_labels_file.readline()

predict_labels

'2 0 1 1 1 1 0 1 0 0 1 0 0 2 0 1 2 2 1 1 1 1 0 0 0 2 2 1 1 1 1 2 2 1 1 1 0 0 2 1 1 0 1 2 2 0 1 1 0 0 2 2 1 1 2 2 2 1 1 2 0 1 2 1 0 0 0 2 0 1 2 0 2 2 0 1 1 1 2 1 1 2 2 2 1 1 1 1 0 0 1 1 1 0 1 0 1 2 1 2 1 1 1 2 2 1 0 2 0 1 0 0 0 1 0 1 1 1 1 1 2 1 2 1 0 0 1 0 0 1 1 0 2 1 0 0 1 0 1 2 1 0 0 1 1 0 0 2 1 1 2 1 0 1 2 1 2 1 1 1 1 0 0 1 1 0 1 0 1 2 1 1 1 2 0 0 2 2 2 2 2 0 2 1 2 1 2 2 2 2 1 2 1 1 2 1 0 1 1 0 1 0 1 1 1 0 2 2 0 1 0 1 1 0 2 2 0 1 1 1 1 0 1 2 1 0 1 0 2 1 1 1 0 1 2 0 1 2 1 0 1 2 2 1 2 0 1 0 2 2 2 0 2 2 0 1 1 2 0 0 2 0 1 1 1 2 1 1 1 0 0 0 1 2 0 2 0 1 2 0 2 1 1 0 1 2 2 1 2 1 2 1 1 2 2 2 0 1 1 1 2 1 1 0 0 0 1 1 2 1 1 1 0 1 2 1 0 2 2 1 1 0 1 1 2 0 1 2 2 0 0 1 1 2 1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 2 1 1 0 2 1 1 0 2 0 1 2 1 1 1 0 1 1 2 1 1 2 1 1 1 1 1 1 2 2 0 0 1 1 1 2 1 2 0 0 0 1 1 1 2 0 0 0 1 2 1 1 2 2 1 2 0 1 2 0 1 1 2 2 2 1 0 1 1 2 2 2 0 1 1 1 2 0 0 0 0 1 0 1 0 1 2 0 0 1 1 1 0 0 0 2 0 0 0 1 1 1 0 0 2 1 0 2 1 2 1 0 1 1 0 1 1 2 1 1 1 1 0 0 1 1 2 0 2 1 1 0 2 0 2 1 0 1 0 2 1 0 1 2 1 1 1 2 2 0 1

In [ ]:
df['predict_label'] = predict_labels.split()
df

,text,label,predict_label
0,Secretário da Fazenda diz que medidas do gover...,2.0,2
1,"Trade hoje: Mesmo acima dos 100 mil, Ibovespa ...",0.0,0
2,"Com alta de preços e crédito escasso, aumenta ...",1.0,1
3,Criptos têm valor? Veja três mitos que confund...,NaN,1
4,“Pelo menos há agora um aceno à disciplina fis...,1.0,1
...,...,...,...
18498,Auditoria do TCU não vê amparo legal em uso de...,NaN,1
18499,Ibovespa tem 5ª queda consecutiva e volta aos ...,NaN,0
18500,Pessoas físicas vencem Wall Street e ação da G...,NaN,2
18501,Sanofi produzirá mais de 100 milhões de doses ...,NaN,2


In [ ]:
df['predict_label'].isna().sum()

0

In [ ]:
df

,text,label,predict_label
0,Secretário da Fazenda diz que medidas do gover...,2.0,2
1,"Trade hoje: Mesmo acima dos 100 mil, Ibovespa ...",0.0,0
2,"Com alta de preços e crédito escasso, aumenta ...",1.0,1
3,Criptos têm valor? Veja três mitos que confund...,NaN,1
4,“Pelo menos há agora um aceno à disciplina fis...,1.0,1
...,...,...,...
18498,Auditoria do TCU não vê amparo legal em uso de...,NaN,1
18499,Ibovespa tem 5ª queda consecutiva e volta aos ...,NaN,0
18500,Pessoas físicas vencem Wall Street e ação da G...,NaN,2
18501,Sanofi produzirá mais de 100 milhões de doses ...,NaN,2


In [ ]:
df.to_csv('datasets/infomoney/infomoney_full_labelled.csv', index=False)

In [ ]:
df.to_csv('datasets/infomoney/infomoney_full_labelled_ROWS.csv', sep='|', index=False)

# Testando modelo no dataset do Kaggle

In [5]:
smn_df = pd.read_csv('datasets/stock_market_news_labelled.csv')
smn_df

,text,y
0,A Technopolis planeja desenvolver em etapas um...,1
1,"A Elcoteq, empresa internacional da indústria ...",0
2,Com a nova planta de produção a empresa aument...,2
3,De acordo com a estratégia atualizada da empre...,2
4,FINANCIAMENTO DO CRESCIMENTO DA ASPOCOMP A Asp...,2
...,...,...
4821,LONDRES MarketWatch - Os preços das ações term...,0
4822,"As vendas de cerveja da Rinkuskiai caíram 6,5 ...",1
4823,"O lucro operacional caiu para EUR 35,4 milhões...",0
4824,As vendas líquidas do segmento de Papel diminu...,0


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [11]:
smn_df.loc[:, 'pred'] = smn_df.loc[:, 'text'].apply(predict_sentiment)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [12]:
smn_df.to_csv('datasets/stock_market_news_prediction.csv', index=False)

In [ ]:
smn_df.read_csv('')

In [13]:
smn_df

,text,y,pred
0,A Technopolis planeja desenvolver em etapas um...,1,2
1,"A Elcoteq, empresa internacional da indústria ...",0,0
2,Com a nova planta de produção a empresa aument...,2,2
3,De acordo com a estratégia atualizada da empre...,2,2
4,FINANCIAMENTO DO CRESCIMENTO DA ASPOCOMP A Asp...,2,2
...,...,...,...
4821,LONDRES MarketWatch - Os preços das ações term...,0,0
4822,"As vendas de cerveja da Rinkuskiai caíram 6,5 ...",1,0
4823,"O lucro operacional caiu para EUR 35,4 milhões...",0,0
4824,As vendas líquidas do segmento de Papel diminu...,0,0


In [14]:
smn_df['error'] = np.abs(smn_df['y'] - smn_df['pred'])
smn_df

,text,y,pred,error
0,A Technopolis planeja desenvolver em etapas um...,1,2,1
1,"A Elcoteq, empresa internacional da indústria ...",0,0,0
2,Com a nova planta de produção a empresa aument...,2,2,0
3,De acordo com a estratégia atualizada da empre...,2,2,0
4,FINANCIAMENTO DO CRESCIMENTO DA ASPOCOMP A Asp...,2,2,0
...,...,...,...,...
4821,LONDRES MarketWatch - Os preços das ações term...,0,0,0
4822,"As vendas de cerveja da Rinkuskiai caíram 6,5 ...",1,0,1
4823,"O lucro operacional caiu para EUR 35,4 milhões...",0,0,0
4824,As vendas líquidas do segmento de Papel diminu...,0,0,0


In [15]:
smn_df['error'].sum()

2415

In [16]:
smn_df['error'].value_counts()

0    2538
1    2161
2     127
Name: error, dtype: int64

In [17]:
smn_df[smn_df['error']==2]

,text,y,pred,error
113,A Nokia também observou que o preço médio de v...,2,0,2
638,"No segundo trimestre de 2010, a perda antes do...",2,0,2
641,A fabricante finlandesa de fibras e produtos p...,2,0,2
644,A siderúrgica finlandesa Rautaruukki Oyj (HEL:...,2,0,2
652,"Em contraste, o prejuízo líquido da empresa no...",2,0,2
...,...,...,...,...
4791,"Graças à Internet, os consumidores comparam pr...",0,2,2
4797,A Qualcomm estimou um lucro no primeiro trimes...,0,2,2
4812,O lucro operacional comparável totalizou EUR 4...,0,2,2
4815,"Nos Estados Bálticos, a empresa reporta vendas...",0,2,2
